<a href="https://colab.research.google.com/github/wonjae124/2021_Lab_project/blob/main/torch_nn_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#모델에 conv나 linear를 1개 추가시 parameter 개수는 2개 증가한다. 왜냐하면 weight와 bias로 총 2개가 증가하기 때문이다.

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16*6*6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features



In [3]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [4]:
for param in net.parameters(): #6,1,3,3은 weight이고 단일 값인 6은 bias이다.
    print(type(param),param.size())

<class 'torch.nn.parameter.Parameter'> torch.Size([6, 1, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([6])
<class 'torch.nn.parameter.Parameter'> torch.Size([16, 6, 3, 3])
<class 'torch.nn.parameter.Parameter'> torch.Size([16])
<class 'torch.nn.parameter.Parameter'> torch.Size([120, 576])
<class 'torch.nn.parameter.Parameter'> torch.Size([120])
<class 'torch.nn.parameter.Parameter'> torch.Size([84, 120])
<class 'torch.nn.parameter.Parameter'> torch.Size([84])
<class 'torch.nn.parameter.Parameter'> torch.Size([10, 84])
<class 'torch.nn.parameter.Parameter'> torch.Size([10])


In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())



10
torch.Size([6, 1, 3, 3])


In [23]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)


tensor([[-0.0690,  0.1088, -0.0685,  0.0086,  0.1180, -0.0991, -0.0494,  0.0592,
          0.0406,  0.0422]], grad_fn=<AddmmBackward0>)


In [24]:
net.zero_grad()
out.backward(torch.randn(1, 10))

#손실함수

In [26]:
input.shape

torch.Size([1, 1, 32, 32])

In [28]:
out = net(input)
out

tensor([[-0.0690,  0.1088, -0.0685,  0.0086,  0.1180, -0.0991, -0.0494,  0.0592,
          0.0406,  0.0422]], grad_fn=<AddmmBackward0>)

In [29]:
out.shape

torch.Size([1, 10])

In [33]:
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
print('loss : ', loss)

loss :  tensor(1.6406, grad_fn=<MseLossBackward0>)


# nn.Linear 
# y =xA^t+b


In [7]:
m = nn.Linear(20,30) # in_feature 20을 out_features 30으로 바꾸겠다.
input = torch.randn(128, 20)
output = m(input) # (128X20) => 128X30 
print(output.size())

n = nn.Linear(20,30) # output. A = 20X30 인데 A^T로 바꾸므로 30X20이 된다. 마지막만 바꿀 기준이 될 수 있다.
input = torch.randn(40,50,20) #input. 마지막만 바꿀 수 있다.
output = n(input)
print(output.size())

k = nn.Linear(10*20,10*30)
input = torch.randn(40,50,200)
output = k(input)
print(output.size())


torch.Size([128, 30])
torch.Size([40, 50, 30])
torch.Size([40, 50, 300])


# TORCH.TENSOR.VIEW

In [8]:
x = torch.randn(4, 4)
x.size()

torch.Size([4, 4])

In [9]:
y = x.view(16)
y.size()

torch.Size([16])

In [10]:
z = x.view(-1,8) # -1은 임의로 크기를 만들어낸다.

In [11]:
z.size()

torch.Size([2, 8])